## Plot loop of clouds from .zarr file

In [14]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import geogif

In [16]:
zarr_path = '/storage/cdalden/goes/goes16/2023/'
zarr_file = '1june.zarr'

ds = xr.open_zarr(zarr_path + zarr_file)
# ds = ds.sel(t='2023-06-01T00:00:00', method='nearest')
ds

<xarray.Dataset> Size: 133MB
Dimensions:                                           (t: 288,
                                                       num_star_looks: 24,
                                                       y: 160, x: 241,
                                                       number_of_harmonization_coefficients: 3,
                                                       number_of_image_bounds: 2,
                                                       band: 1,
                                                       number_of_time_bounds: 2)
Coordinates:
    band_wavelength_star_look                         (num_star_looks) float32 96B dask.array<chunksize=(24,), meta=np.ndarray>
    y_image                                           float32 4B ...
  * y                                                 (y) float64 1kB 0.1092 ...
    t_star_look                                       (num_star_looks) datetime64[ns] 192B dask.array<chunksize=(24,), meta=np.ndarray>
    x_image                                           float32 4B ...
    band_wavelength                                   (band) float32 4B dask.array<chunksize=(1,), meta=np.ndarray>
    band_id                                           (band) int8 1B dask.array<chunksize=(1,), meta=np.ndarray>
  * t                                                 (t) datetime64[ns] 2kB ...
  * x                                                 (x) float64 2kB -0.0746...
Dimensions without coordinates: num_star_looks,
                                number_of_harmonization_coefficients,
                                number_of_image_bounds, band,
                                number_of_time_bounds
Data variables: (12/39)
    focal_plane_temperature_threshold_increasing      (t) float32 1kB dask.array<chunksize=(288,), meta=np.ndarray>
    nominal_satellite_height                          (t) float32 1kB dask.array<chunksize=(288,), meta=np.ndarray>
    Rad                                               (t, y, x) float64 89MB dask.array<chunksize=(72, 40, 121), meta=np.ndarray>
    a_h_NRTH                                          (t, number_of_harmonization_coefficients) float32 3kB dask.array<chunksize=(288, 3), meta=np.ndarray>
    nominal_satellite_subpoint_lat                    (t) float32 1kB dask.array<chunksize=(288,), meta=np.ndarray>
    esun                                              (t) float32 1kB dask.array<chunksize=(288,), meta=np.ndarray>
    ...                                                ...
    maximum_focal_plane_temperature                   (t) float32 1kB dask.array<chunksize=(288,), meta=np.ndarray>
    valid_pixel_count                                 (t) float64 2kB dask.array<chunksize=(288,), meta=np.ndarray>
    yaw_flip_flag                                     (t) float32 1kB dask.array<chunksize=(288,), meta=np.ndarray>
    b_h_NRTH                                          (t, number_of_harmonization_coefficients) float32 3kB dask.array<chunksize=(288, 3), meta=np.ndarray>
    max_radiance_value_of_valid_pixels                (t) float32 1kB dask.array<chunksize=(288,), meta=np.ndarray>
    planck_bc1                                        (t) float32 1kB dask.array<chunksize=(288,), meta=np.ndarray>
Attributes: (12/30)
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    standard_name_vocabulary:  CF Standard Name Table (v35, 20 July 2016)
    institution:               DOC/NOAA/NESDIS > U.S. Department of Commerce,...
    project:                   GOES
    production_site:           RBU
    ...                        ...
    timeline_id:               ABI Mode 6
    date_created:              2023-06-01T00:04:03.8Z
    time_coverage_start:       2023-06-01T00:01:17.4Z
    time_coverage_end:         2023-06-01T00:03:55.8Z
    LUT_Filenames:             SpaceLookParams(FM1A_ADR1270_PR_12_02_01)-7371...
    id:                        5b93f84c-4b24-4ede-b8b8-3f29dc9e128a

In [17]:
variable = 'Rad'
# select our variable of interest
da = ds[variable]

# create the gif animation
gif_bytes = geogif.dgif(
    da,
    fps=5,
    cmap="Greys_r",
    date_format="%Y-%m-%d %H:%M:%S",
    date_position="ul",
    bytes=True,
).compute()

# write gif to file
with open(f"test1.gif", "wb") as f:
    f.write(gif_bytes)
